# chromVAR

`chromVAR` is a method for computing differential motif accessibility. Its original implementation is [available in Bioconductor](https://bioconductor.org/packages/chromVAR), and it has been described in [Schep et al., 2017](https://www.nature.com/articles/nmeth.4401).

## Prepare a demo object

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from anndata import AnnData
import chame as ch

In [3]:
from chame.tl.chromvar import _get_background_peaks, _compute_expectations_core, _compute_deviations_core

In [4]:
np.random.seed(1)
x = np.random.binomial(5, 0.2, size=(100,10))  # cell x peaks

adata = AnnData(x)
adata.obs_names = [f"cell{i+1}" for i in range(adata.n_obs)]
adata.var_names = [f"peak{j+1}" for j in range(adata.n_vars)]

In [5]:
# Simulate sequences
adata.var["length"] = np.random.choice(np.arange(10, 100), size=adata.n_vars)
adata.var["sequence"] = [''.join(np.random.choice(["A", "C", "G", "T"], size=i)) for i in adata.var.length]

In [6]:
# Compute some peak statistics
adata.var["gc"] = ch.util.seq.count_gc(adata.var["sequence"])
adata.var["n_fragments"] = adata.X.sum(axis=0)

In [7]:
adata

AnnData object with n_obs × n_vars = 100 × 10
    var: 'length', 'sequence', 'gc', 'n_fragments'

In [8]:
# Simulate motif scanning results
n_motifs = 5
motifs_in_peaks = np.random.binomial(1, .3, size=(n_motifs, adata.n_vars))

## Run chromVAR

In [27]:
ch.tl.chromvar(adata, motifs_in_peaks, bias="gc")

/usr/local/opt/python@3.8/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/chame/tl/chromvar.py:147: RuntimeWarning: invalid value encountered in true_divide
  z[fail_filter] = np.nan
2022-06-04 14:55:29.565 | INFO     | chame.tl.chromvar:chromvar:62 - Added key deviations to adata.obsm.
2022-06-04 14:55:29.566 | INFO     | chame.tl.chromvar:chromvar:65 - Added key z to adata.obsm.


In [26]:
adata.obsm

AxisArrays with keys: deviations, z